In [1]:
import requests
import json
import dotenv
import os
from groq import Groq

dotenv.load_dotenv()

OPENROUTER_API_KEY = os.environ["OPENROUTER_KEY"]
GROQ_KEY = os.environ["GROQ_KEY"]

client = Groq(
    api_key=os.environ.get("GROQ_KEY")
)
def call_groq(message):
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"{message}"
            }
        ],
        model=  "llama3-70b-8192"
    )
    
    return chat_completion

def get_agent_response(message):
    response = call_groq(message)
    return response.choices[0].message.content

In [2]:
print(response.choices[0].message.content)

Here's the calculation:

1 + 3 = 4
4 + 5 = 9

So, 1 + 3 + 5 = 9.


In [3]:
import json
with open("gpt-crawler/output-1.json", "r") as f:
    data = json.load(f)

In [4]:
content_map = {}
titles = []
for topic in data:
    titles.append(topic['title'])
    content_map[topic['title']] = topic['html']
        

In [6]:
#idea: (relevanceAI) -> templater -> ex_templater
def templater(title, content):
    return f"""
    Given the following documentation, please first summarize step by step what the section is about and then provide a code snippet that demonstrates the functionality of the section.
    ################################################
    You are given the title: {title}
    ################################################
    Here is the documentation:
    {content}
    """
    
def ex_templater(summary):
    return f"""
    Given the following summary, please write a coding exercise for the user to complete. You should write most of the code out already, and only leave a few lines blank for the user to write out
    
    You should only generate the instructions and the code exercise. The solutions and explanations are to be omitted. The focus on the exercises/incomplete lines should be on the code related to the given documentation
    
    The following example is an example of what you should output:
   Here are a few coding exercises based on the summary:

################################################
Exercise 1:
Write code to compute the inverse discrete Fourier transform of a 1D signal using `torch.fft.ifft()`.

```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

###TODO: Compute the inverse discrete Fourier transform of the signal (roughly 1 line)

print(ifft_signal)
```

################################################
Exercise 2:
Write code to compute the 2D discrete Fourier transform of a 2D signal using `torch.fft.fftn()`.

```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

###TODO: Compute the 2D discrete Fourier transform of the signal (roughly 1 line)

print(fftn_signal)
```

################################################
Exercise 3:
Write code to shift the zero-frequency component of a 1D signal to the center of the spectrum using `torch.fft.fftshift()`.

```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

###TODO: Shift the zero-frequency component of the signal to the center of the spectrum (roughly 1 line)

print(shifted_fft_signal)
```

Note: You need to complete the exercises by filling in the `###TODO` lines with the appropriate code.
    Here is the summary:
    {summary}
    """


for i in titles[:10]:
    topic_ex = content_map[i]
    template = templater(i, topic_ex)
    res = get_agent_response(template)
    
    summary = res.choices[0].message.content
    ex_template = ex_templater(summary)
    ex = get_agent_response(ex_template)    
    break


Here are a few coding exercises based on the summary:

################################################
Exercise 1:
Write code to perform a 1D discrete Fourier transform of a signal using `torch.fft.fft()`.

```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

###TODO: Compute the discrete Fourier transform of the signal (roughly 1 line)

print(fft_signal)
```

################################################
Exercise 2:
Write code to compute the 2D discrete Fourier transform of a 2D signal using `torch.fft.fftn()`.

```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

###TODO: Compute the 2D discrete Fourier transform of the signal (roughly 1 line)

print(fftn_signal)
```

################################################
Exercise 3:
Write code to compute the sample frequencies for a 1D signal using `torch.fft.fftfreq()`.

```python
import torch

# Create a sample signal
signal = torch

In [6]:
def answer_template(exercise):
    return f"""
    Given the following coding exercise, please write out the code that would complete the exercise. You should write out the code that would complete the exercise.
    Ensure the solution works for the test cases, and that the test cases align with the solution code.
    Example solution will be:
    Here are the completed exercises with test cases:

**Exercise 1**
```python
import torch
import numpy as np

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8], dtype=torch.float32)

# Compute the sample frequencies associated with the Fourier transform of the signal
freq = torch.fft.fftfreq(signal.size()[0], dtype=torch.float32)

print(freq)

**Testcase 1**
assert freq.shape == torch.Size([8])
assert torch.allclose(freq, torch.tensor([-0.5, -0.375, -0.25, -0.125, 0.125, 0.25, 0.375, 0.5]))
```

**Exercise 2**
```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute the 2D discrete Fourier transform of the signal
fftn_signal = torch.fft.fftn(signal)

# Reorder the elements of the FFT data
reordered_fftn_signal = torch.fft.fftshift(fftn_signal)

print(reordered_fftn_signal)

**Testcase 2**
assert reordered_fftn_signal.shape == torch.Size([3, 3])
assert torch.allclose(reordered_fftn_signal, torch.tensor([[7.06225781e+01+1.22464680e-16j, 2.12132034e+00-1.22464680e-16j, 2.12132034e+00+1.22464680e-16j], 
                                                            [2.12132034e+00-1.22464680e-16j, 3.53553391e+00+1.22464680e-16j, 3.53553391e+00-1.22464680e-16j], 
                                                            [2.12132034e+00+1.22464680e-16j, 3.53553391e+00-1.22464680e-16j, 7.06225781e+01-1.22464680e-16j]]))
```

**Exercise 3**
```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute the 2D discrete Fourier transform of the signal
fftn_signal = torch.fft.fftn(signal)

# Compute the inverse 2D discrete Fourier transform of the signal
ifftn_signal = torch.fft.ifftn(fftn_signal)

print(ifftn_signal)

**Testcase 3**
assert ifftn_signal.shape == torch.Size([3, 3])
assert torch.allclose(ifftn_signal, signal)
```
    
    Please also generate test cases for the code that you write. You should ensure that the testcases work. You need to make sure other than the incomplete line, the rest of the previous template code is unchanged. 
    
    {exercise}

"""

ans_temp = answer_template(ex)
ans = get_agent_response(ans_temp)

Here are the completed exercises with test cases:

**Exercise 1**
```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

print(fft_signal)

**Testcase 1**
assert fft_signal.shape == torch.Size([8])
assert torch.allclose(fft_signal, torch.tensor([-2.82842712+2.41421356j, -7.07106781+2.41421356j, -2.82842712+0.41421356j, -2.82842712-0.41421356j, -7.07106781-2.41421356j, -2.82842712-2.41421356j, -7.07106781+2.41421356j, -2.82842712+0.41421356j]))
```

**Exercise 2**
```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute the 2D discrete Fourier transform of the signal
fftn_signal = torch.fft.fftn(signal)

print(fftn_signal)

**Testcase 2**
assert fftn_signal.shape == torch.Size([3, 3])
assert torch.allclose(fftn_signal, torch.tensor([[7.06225781e+01+1.22464680e-16j, 2.12132034e+00-1.22464680e-1

In [7]:
import re

# Example content from the notebook
content = """<insert content from the 'ex' and 'ans' variables here>"""

# Define regex patterns
exercise_pattern = r'Exercise (\d+):\s*([\s\S]*?)(?=Exercise \d+:|$)'
solution_pattern = r'Exercise (\d+)\s*\*\*\n```python\n([\s\S]*?)(?=```)'
testcase_pattern = r'Testcase (\d+)\*\*\n([\s\S]*?)(?=```)'

# Extract exercises
exercises = re.findall(exercise_pattern, exercise)

# Extract solutions
solutions = re.findall(solution_pattern, answer)

# Extract test cases
testcases = re.findall(testcase_pattern, answer)


In [8]:
""" def verifier_template(message, type):
    return f'''There is an error in the answer and exercise templates. Check if it is a template error and re-output the correct template.
    Check if the message below has the correct format.
    Based on the type, we will be looking for if the data matches a specific regex. I also want to ensure you only modify the format to follow the regex, and not the content of the message.
    
    For exercises:
    r'Exercise (\d+):\s*([\s\S]*?)(?=Exercise \d+:|$)'ArithmeticError
    
    For solutions:
    r'Exercise (\d+)\s*\*\*\n```python\n([\s\S]*?)(?=```)'
    r'Testcase (\d+)\*\*\n([\s\S]*?)(?=```)'
    
    ################################################################
    Here is the message:
    Type: {type}
    {message}


    '''

MAX_RETRIES = 3
retries = 0

while not (len(exercises) == len(solutions) == len(testcases)) and retries < MAX_RETRIES:
    retries+=1
    print("Format invalid, redoing template!")
    ex_ver = verifier_template(exercise, "Exercise")
    sol_ver = verifier_template(answer, "Solution")
    exercise = get_agent_response(ex_ver)
    answer = get_agent_response(sol_ver)
    # Extract exercises
    exercises = re.findall(exercise_pattern, exercise)

    # Extract solutions
    solutions = re.findall(solution_pattern, answer)

    # Extract test cases
    testcases = re.findall(testcase_pattern, answer)

 """

' def verifier_template(message, type):\n    return f\'\'\'There is an error in the answer and exercise templates. Check if it is a template error and re-output the correct template.\n    Check if the message below has the correct format.\n    Based on the type, we will be looking for if the data matches a specific regex. I also want to ensure you only modify the format to follow the regex, and not the content of the message.\n    \n    For exercises:\n    r\'Exercise (\\d+):\\s*([\\s\\S]*?)(?=Exercise \\d+:|$)\'ArithmeticError\n    \n    For solutions:\n    r\'Exercise (\\d+)\\s*\\*\\*\n```python\n([\\s\\S]*?)(?=```)\'\n    r\'Testcase (\\d+)\\*\\*\n([\\s\\S]*?)(?=```)\'\n    \n    ################################################################\n    Here is the message:\n    Type: {type}\n    {message}\n\n\n    \'\'\'\n\nMAX_RETRIES = 3\nretries = 0\n\nwhile not (len(exercises) == len(solutions) == len(testcases)) and retries < MAX_RETRIES:\n    retries+=1\n    print("Format invalid,

In [9]:
import mercury as mr

mr.Confetti()

In [12]:
import nbformat as nbf

# Create a new notebook
nb = nbf.v4.new_notebook()


nb.cells.append(nbf.v4.new_markdown_cell("# Setup\nImport your libraries here"))
nb.cells.append(nbf.v4.new_code_cell("import mercury as mr"))

# Loop through the exercises, solutions, and test cases
for ex_number, ex_content in exercises:
    # Add exercise description as a markdown cell
    desc_pattern = r'([\s\S]*?)(?=```python)'
    desc_match = re.search(desc_pattern, ex_content)
    description = desc_match.group(1).strip() if desc_match else ""
    nb.cells.append(nbf.v4.new_markdown_cell(f"# Exercise {ex_number}\n{description.strip()}"))
    
    # Extract code part from exercise content (assume it's inside triple backticks)
    code_match = re.search(r'```python\n([\s\S]*?)\n```', ex_content)
    if code_match:
        exercise_code = code_match.group(1)
        # Add exercise code as a code cell
        nb.cells.append(nbf.v4.new_code_cell(exercise_code.strip()))
    
    # Add hidden solution as a code cell with a docstring to prevent execution
    solution = next((sol[1] for sol in solutions if sol[0] == ex_number), "")
    if solution:
        hidden_solution = f'"""\n# Solution hidden\n{solution.strip()}\n"""'
        solution_cell = nbf.v4.new_code_cell(hidden_solution)
        solution_cell.metadata.update({"tags": ["hide_cell"]})
        nb.cells.append(nbf.v4.new_markdown_cell(f'# Solution {ex_number}'))
        nb.cells.append(solution_cell)
    
    # Add test case as a code cell
    testcase = next((tc[1] for tc in testcases if tc[0] == ex_number), "")
    if testcase:
        testcase_cell = nbf.v4.new_code_cell(testcase.strip()+'\n'+'mr.Confetti()')
        testcase_cell.metadata.update({"tags": ["hide_cell"]})
        nb.cells.append(testcase_cell)

# Save the new notebook
with open('new_exercise_notebook.ipynb', 'w') as f:
    nbf.write(nb, f)


Traceback (most recent call last):
  File "/home/spooky/Documents/Github Projects/codedocs/docs/bin/jupyter-nbconvert", line 10, in <module>
    sys.exit(main())
  File "/home/spooky/Documents/Github Projects/codedocs/docs/lib/python3.10/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/home/spooky/Documents/Github Projects/codedocs/docs/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/spooky/Documents/Github Projects/codedocs/docs/lib/python3.10/site-packages/nbconvert/nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "/home/spooky/Documents/Github Projects/codedocs/docs/lib/python3.10/site-packages/nbconvert/nbconvertapp.py", line 586, in convert_notebooks
    self.exporter = cls(config=self.config)
  File "/home/spooky/Documents/Github Projects/codedocs/docs/lib/python3.10/site-packages/nbconvert/exporters/t

<IPython.core.display.Javascript object>